In [ ]:
! pip install streamlit -q

     |████████████████████████████████| 10.1 MB 5.3 MB/s 
     |████████████████████████████████| 181 kB 10.7 MB/s 
     |████████████████████████████████| 4.3 MB 6.3 MB/s 
     |████████████████████████████████| 111 kB 4.0 MB/s 
     |████████████████████████████████| 164 kB 44.5 MB/s 
     |████████████████████████████████| 76 kB 4.4 MB/s 
     |████████████████████████████████| 63 kB 679 kB/s 
     |████████████████████████████████| 131 kB 42.4 MB/s 
     |████████████████████████████████| 793 kB 17.2 MB/s 
     |████████████████████████████████| 428 kB 45.1 MB/s 
     |████████████████████████████████| 130 kB 49.7 MB/s 
     |████████████████████████████████| 381 kB 42.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1.0

In [ ]:
!pip install lime shap

     |████████████████████████████████| 275 kB 5.2 MB/s 
     |████████████████████████████████| 564 kB 51.6 MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=b3bf758ea800376525836b5d491ef24ccbe3699dcc4522ce69b66824eaf96735
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built lime


In [ ]:
from google.colab import drive
import pandas as pd 

drive.mount('/content/drive')

census_train = pd.read_csv(filepath_or_buffer="/content/drive/MyDrive/census-income.data")
census_test = pd.read_csv(filepath_or_buffer="/content/drive/MyDrive/census-income.test")

Mounted at /content/drive


In [ ]:
%%writefile app.py
from matplotlib import axes
import matplotlib.pyplot as plt
import streamlit as st
from sklearn.datasets import make_classification
import lime
import lime.lime_tabular
from lime import lime_tabular
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import svm
import altair as alt
import pandas as pd
import shap
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from google.colab import drive

st.set_option('deprecation.showPyplotGlobalUse', False)

page_names = ['Synthetic', 'Real World']
page = st.radio('Data Type', page_names)

# ALL CODE BELOW WILL BE FOR SYNTETIC
if page == 'Synthetic':

  # Create interactive inputs for num features, redundant, repetitive, etc.
  num_samples = st.slider('Choose the number of samples', 0, 20000, 1000)

  num_feats = st.slider('Number of features', 
                  min_value= int((math.log(4)/math.log(2))), 
                  max_value=100, 
                  value=4)
  num_inform = st.slider('Number of informative features', 
                  min_value=0, 
                  max_value=num_feats, 
                  value=2)
  num_red = st.slider('Number of redundant features', 
                  min_value=0, 
                  max_value=(num_feats - num_inform), 
                  value=1)
  num_rep = st.slider('Number of repetitive features', 
                  min_value=0, 
                  max_value= (num_feats - num_inform - num_red), 
                  value=1)
  flip_val = st.slider('Flip_y value', 
                  min_value=-1.0, 
                  max_value= 1.0, 
                  value=.01)
  weights_val = st.slider('Binary Classification weight for class 0', 
                  min_value=0.0, 
                  max_value= 1.0, 
                  value=.50)

  # Create the synthetic data
  x,y = make_classification(n_samples=num_samples, 
                            n_features= num_feats,
                            n_informative= num_inform, 
                            n_redundant= num_red, 
                            n_repeated= num_rep, 
                            n_classes=2, 
                            n_clusters_per_class=2, 
                            weights= [weights_val, 1 - weights_val], 
                            flip_y= flip_val, 
                            class_sep=1.0, 
                            hypercube=True, 
                            shift=0.0, 
                            scale=1.0, 
                            shuffle=True, 
                            random_state=2022)


  #load the synthetic data to web-app
  def load_x_data(nrows):
      data = x[0:nrows]
      return data

  def load_y_data(nrows):
      data = y[0:nrows]
      return data
    
  x_data = load_x_data(1000)
  y_data = load_y_data(1000)


  #write scatter plot of synthetic data to web-app
  feature_to_filter_one = st.slider('Choose x-axis feature', 0, x_data.shape[1] - 1, 0)
  feature_to_filter_two = st.slider('Choose y-axis feature', 0, x_data.shape[1] - 1, 1)

  source = pd.DataFrame(x_data)
  y_source = pd.DataFrame(y_data)
  df_new = source.iloc[:, [feature_to_filter_one,feature_to_filter_two]]
  df_new['output'] = y_source
  df_new.columns=[(f'Feature {feature_to_filter_one}'),(f'Feature {feature_to_filter_two}'), 'output']

  st.subheader(f'Synthetic Data Scatter Plot of Features {feature_to_filter_one} and {feature_to_filter_two}')

  st.write(alt.Chart(df_new).mark_circle(size=60).encode(
      x=(f'Feature {feature_to_filter_one}'),
      y=(f'Feature {feature_to_filter_two}'),
      color='output',
      tooltip=[(f'Feature {feature_to_filter_one}'), (f'Feature {feature_to_filter_two}'), 'output']
  ).interactive())


  #write the ML model to the web-app
  X_train, X_test, y_train, y_test = train_test_split(source, y_source, test_size=0.2)


  clf = svm.SVC(probability=True)
  clf.fit(X_train, y_train)

  for i in [X_test]:
    y_guess = clf.predict_proba(X_test)


  # Write the SHAP Plots into the web-app
  st.subheader('SHAP Plots')

  shap_graph = st.checkbox('Generate SHAP Summary Plot')

  if shap_graph:
    st.write('Calculating SHAP values...')
    test_data = shap.sample(X_test, 15)
    explainer = shap.KernelExplainer(clf.predict_proba, X_train[:15])
    shap_values = explainer.shap_values(test_data)
    st.write('SHAP values calculated!')
    sum_plot = shap.summary_plot(shap_values, max_display=20)
    st.pyplot(sum_plot)

    st.subheader(f'Scatter Plot of SHAP Values of Features {feature_to_filter_one} and {feature_to_filter_two}')

    shap_feature_one_class1 = []
    shap_feature_two_class1 = []

    shap_feature_one_class2 = []
    shap_feature_two_class2 = []

    for i in range(len(shap_values[0])):
      shap_feature_one_class1.append(shap_values[0][i][feature_to_filter_one])
      shap_feature_two_class1.append(shap_values[0][i][feature_to_filter_two])

    for i in range(len(shap_values[1])):
      shap_feature_one_class2.append(shap_values[1][i][feature_to_filter_one])
      shap_feature_two_class2.append(shap_values[1][i][feature_to_filter_two])

    merged = []

    for i in range(len(shap_feature_one_class1)):
      x = {f'Feature {feature_to_filter_one}': shap_feature_one_class1[i], f'Feature {feature_to_filter_two}': shap_feature_two_class1[i],'Class':0}
      merged.append(x)
      y = {f'Feature {feature_to_filter_one}': shap_feature_one_class2[i], f'Feature {feature_to_filter_two}': shap_feature_two_class2[i],'Class':1}
      merged.append(y)

    df_shap = pd.DataFrame(merged)

    st.write(alt.Chart(df_shap).mark_circle(size=60).encode(
        x = alt.X(f'Feature {feature_to_filter_one}'),
        y = alt.Y(f'Feature {feature_to_filter_two}'),
        color=alt.Color('Class:N', legend=alt.Legend(title=("Class"))),
    ).interactive())


  # Write the LIME Plot into the Web-APP
  st.subheader("LIME Plots")

  sample = st.slider('Pick an instance to analyze', 0, len(X_test), int(len(X_test)/5))

  lime_graph = st.checkbox("Show LIME Graph")

  col_names = []
  for i in range(source.shape[1]):
    col_names.append(f'feature {i}')

  if lime_graph:
    lime_explainer = lime.lime_tabular.LimeTabularExplainer(
        training_data=X_train.values,
        feature_names=col_names,
        class_names=["Class 0","Class 1"],
        mode="classification",
        verbose=True,
        random_state=2022
    )

    num = sample
    test_sample = X_test.iloc[num,:]
    exp = lime_explainer.explain_instance(test_sample, predict_fn=clf.predict_proba, num_features=20, num_samples=100)
    st.pyplot(exp.as_pyplot_figure(), clear_figure=False)
    lime_graph = False



# ALL CODE BELOW FOR REAL WORLD DATA
else:
  census_train = pd.read_csv(filepath_or_buffer="/content/drive/MyDrive/census-income.data")
  census_test = pd.read_csv(filepath_or_buffer="/content/drive/MyDrive/census-income.test")


  census_train = census_train.sample(n=80000,random_state=2)
  census_test = census_test.sample(n=20000,random_state=2)


  census_train.columns = census_test.columns = ['age','class_of_worker','detailed_industry_recode','detailed_occupation_recode','education','wage_per_hour','enrolled_in_edu_inst_last_wk',
                          'marital_stat','major_industry_code','major_occupation_code','race','hispanic_origin','sex','member_of_a_labor_union','reason_for_unemployment',
                          'full_or_part_time_employment_stat','capital_gains','capital_losses','dividends_from_stocks','tax_filer_stat','region_of_previous_residence',
                          'state_of_previous_residence','detailed_household_and_family_stat','detailed_household_summary_in_household','instance_weight_ignore','instance_weight','migration_code-change_in_msa',
                          'migration_code-change_in_reg','migration_code-move_within_reg','live_in_this_house_1_year_ago','migration_prev_res_in_sunbelt','num_persons_worked_for_employer',
                          'family_members_under_18','country_of_birth_father','country_of_birth_mother','country_of_birth_self','citizenship','own_business_or_self_employed',
                          'fill_inc_questionnaire_for_veterans_admin','veterans_benefits','weeks_worked_in_year','year']

  census_train = census_train[census_train.wage_per_hour != 0]
  census_test = census_test[census_test.wage_per_hour != 0]
  
  mean = census_train["wage_per_hour"].mean()
  census_train['wage_per_hour'] = census_train['wage_per_hour'].apply(lambda x: 1 if x > mean else 0)
  census_test['wage_per_hour'] = census_test['wage_per_hour'].apply(lambda x: 1 if x > mean else 0)

  options = []

  columns_to_aggregate = st.multiselect(
    'What features would you like to use?',
      ['age','class_of_worker','education','major_industry_code','major_occupation_code','member_of_a_labor_union','capital_gains','capital_losses','dividends_from_stocks','weeks_worked_in_year'])
  
  options = columns_to_aggregate

  if len(options) >= 2:
    feature_to_filter_one = st.radio('Pick an x-axis feature', options, index=0)
    feature_to_filter_two = st.radio('Pick an y-axis feature', options, index=1)

    scat_df = census_train[[feature_to_filter_one,feature_to_filter_two,'wage_per_hour']]
    class_color = []
    for i in scat_df['wage_per_hour']:
      if i == 0:
        class_color.append('Class 0')
      else:
        class_color.append('Class 1')
    
    scat_df['Class'] = class_color

    st.subheader((f'{feature_to_filter_one} vs {feature_to_filter_two}'))
    st.subheader((f'Class 0: Earns below {int(mean)} per day'))
    st.subheader((f' Class 1: Earns above {int(mean)} per day'))

    st.write(alt.Chart(scat_df).mark_circle(size=30).encode(
        x=(f'{feature_to_filter_one}'),
        y=(f'{feature_to_filter_two}'),
        color=alt.Color('Class', legend=alt.Legend(title=("Wage Class"))),
        tooltip=[(f'{feature_to_filter_one}'), (f'{feature_to_filter_two}'), 'Class']
        ).properties(
          width=800,
          height=400
        ).interactive())

    if len(options) > 0:    
      column_names = options
      column_names.append('wage_per_hour')


      #training set
      census_train = census_train[column_names]

      le = LabelEncoder() # label encoder 
      if 'class_of_worker' in options:
        census_train['class_of_worker'] = le.fit_transform(census_train['class_of_worker'])
      if 'education' in options:
        census_train['education'] = le.fit_transform(census_train['education'])
      if 'major_industry_code' in options:
        census_train['major_industry_code'] = le.fit_transform(census_train['major_industry_code'])
      if 'major_occupation_code' in options:
        census_train['major_occupation_code'] = le.fit_transform(census_train['major_occupation_code'])
      if 'member_of_a_labor_union' in options:
        census_train['member_of_a_labor_union'] = le.fit_transform(census_train['member_of_a_labor_union'])
      
      le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

      X_train = census_train.drop(['wage_per_hour'], axis=1)
      col_names = X_train.columns.values

      s = MinMaxScaler()
      X_train[X_train.columns] = s.fit_transform(X_train[X_train.columns])

      y_train = census_train['wage_per_hour']

      clf = svm.SVC(probability=True)
      clf.fit(X_train, y_train)


      #testing set
      census_test = census_test[column_names]

      le = LabelEncoder() # label encoder 
      if 'class_of_worker' in options:
        census_test['class_of_worker'] = le.fit_transform(census_test['class_of_worker'])
      if 'education' in options:
        census_test['education'] = le.fit_transform(census_test['education'])
      if 'major_industry_code' in options:
        census_test['major_industry_code'] = le.fit_transform(census_test['major_industry_code'])
      if 'major_occupation_code' in options:
        census_test['major_occupation_code'] = le.fit_transform(census_test['major_occupation_code'])
      if 'member_of_a_labor_union' in options:
        census_test['member_of_a_labor_union'] = le.fit_transform(census_test['member_of_a_labor_union'])

      X_test = census_test.drop(['wage_per_hour'], axis=1)
      col_names = X_test.columns.values

      s = MinMaxScaler()
      X_test[X_test.columns] = s.fit_transform(X_test[X_test.columns])

      y_test = census_test['wage_per_hour']

      clf = svm.SVC(probability=True)
      clf.fit(X_test, y_test)


      # Write the SHAP Plots into the web-app
      st.subheader('SHAP Plots')

      shap_graph = st.checkbox('Generate SHAP Summary Plot')

    if shap_graph:
    
     
      st.write('Calculating SHAP values...')
      test_data = shap.sample(X_test, 15) 
      explainer = shap.KernelExplainer(clf.predict_proba, X_train[:15]) #gets first 15 records of X_train
      shap_values = explainer.shap_values(test_data)
      st.write('SHAP values calculated!')
      sum_plot = shap.summary_plot(shap_values, feature_names = options)
      st.pyplot(sum_plot)


      st.subheader(f'Scatter Plot of SHAP Values of Features {feature_to_filter_one} and {feature_to_filter_two}')

      shap_feature_one_class1 = []
      shap_feature_two_class1 = []

      shap_feature_one_class2 = []
      shap_feature_two_class2 = []

      for i in range(len(shap_values[0])):
        shap_feature_one_class1.append(shap_values[0][i][options.index(feature_to_filter_one)])
        shap_feature_two_class1.append(shap_values[0][i][options.index(feature_to_filter_two)])

      for i in range(len(shap_values[1])):
        shap_feature_one_class2.append(shap_values[1][i][options.index(feature_to_filter_one)])
        shap_feature_two_class2.append(shap_values[1][i][options.index(feature_to_filter_two)])

      merged = []

      for i in range(len(shap_feature_one_class1)):
        x = {f'Feature {feature_to_filter_one}': shap_feature_one_class1[i], f'Feature {feature_to_filter_two}': shap_feature_two_class1[i],'Class':0}
        merged.append(x)
        y = {f'Feature {feature_to_filter_one}': shap_feature_one_class2[i], f'Feature {feature_to_filter_two}': shap_feature_two_class2[i],'Class':1}
        merged.append(y)

      df_shap = pd.DataFrame(merged)

      st.write(alt.Chart(df_shap).mark_circle(size=60).encode(
          x = alt.X(f'Feature {feature_to_filter_one}'),
          y = alt.Y(f'Feature {feature_to_filter_two}'),
          color=alt.Color('Class:N', legend=alt.Legend(title=("Class"))),
      ).interactive())



      # Write the LIME Plot into the Web-APP
      st.subheader("LIME Plots")

      sample = st.slider('Pick an instance to analyze', 0, len(X_test), int(len(X_test)/5))

      lime_graph = st.checkbox("Show LIME Graph")

      if lime_graph:
        lime_explainer = lime_tabular.LimeTabularExplainer(
            training_data=X_train.values,
            feature_names=options,
            class_names=[(f"Wage Above {mean}"),(f"Wage Below {mean}")],
            mode="classification",
            verbose=True,
            random_state=2022
          )
        num = 2
        test_sample = X_test.iloc[num,:]
        exp = lime_explainer.explain_instance(test_sample, predict_fn=clf.predict_proba, num_features=10, num_samples=100)
        st.pyplot(exp.as_pyplot_figure(), clear_figure=False)
        lime_graph = False
        st.write(le_name_mapping)
      

Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

2022-05-13 01:49:41.368 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 3.897s
your url is: https://nice-crews-carry-35-231-246-201.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.231.246.201:8501


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on